# Hindsight Quickstart

This notebook covers the basics of using Hindsight:
- **Retain**: Store information in memory
- **Recall**: Retrieve memories matching a query
- **Reflect**: Generate insights from memories

## Prerequisites

Make sure you have Hindsight running. The easiest way is via Docker:

```bash
export OPENAI_API_KEY=your-key

docker run --rm -it --pull always -p 8888:8888 -p 9999:9999 \
  -e HINDSIGHT_API_LLM_API_KEY=$OPENAI_API_KEY \
  -e HINDSIGHT_API_LLM_MODEL=o3-mini \
  -v $HOME/.hindsight-docker:/home/hindsight/.pg0 \
  ghcr.io/vectorize-io/hindsight:latest
```

- API: http://localhost:8888
- UI: http://localhost:9999

## Installation

Install the Hindsight Python client:

In [24]:
!pip install hindsight-client nest_asyncio python-dotenv -U

## Connect to Hindsight

In [25]:
# Jupyter notebooks already run an asyncio event loop. The hindsight client 
# uses loop.run_until_complete() internally, but Python doesn't allow nested 
# event loops by default. nest_asyncio patches this to allow nesting.
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv

# Load environment variables from .env file
# Copy .env.example to .env and fill in your values
load_dotenv()

# Configuration (override with env vars if set)
HINDSIGHT_API_URL = os.getenv("HINDSIGHT_API_URL", "http://localhost:8888")
HINDSIGHT_UI_URL = os.getenv("HINDSIGHT_UI_URL", "http://localhost:9999")

from hindsight_client import Hindsight

client = Hindsight(base_url=HINDSIGHT_API_URL)

## Retain: Store Information

The `retain` operation is used to push new memories into Hindsight. It tells Hindsight to _retain_ the information you pass in.

Behind the scenes, the retain operation uses an LLM to extract key facts, temporal data, entities, and relationships.

In [26]:
# Simple retain
client.retain(
    bank_id="my-bank",
    content="Alice works at Google as a software engineer"
)

# View the stored document in the UI:
print(f"View documents: {HINDSIGHT_UI_URL}/banks/my-bank?view=documents")

View documents: http://localhost:9999/banks/my-bank?view=documents


In [27]:
# Retain with context and timestamp
client.retain(
    bank_id="my-bank",
    content="Alice got promoted to senior engineer",
    context="career update",
    timestamp="2025-06-15T10:00:00Z"
)

RetainResponse(success=True, bank_id='my-bank', items_count=1, var_async=False)

## Recall: Retrieve Memories

The `recall` operation retrieves memories matching a query. It performs 4 retrieval strategies in parallel:
- **Semantic**: Vector similarity
- **Keyword**: BM25 exact matching
- **Graph**: Entity/temporal/causal links
- **Temporal**: Time range filtering

In [28]:
# Simple recall
results = client.recall(bank_id="my-bank", query="What does Alice do?")

print("Memories:")
for r in results.results:
    print(f"  - {r.text}")

Memories:
  - Alice is believed by the user to be a dedicated and skilled professional who has demonstrated significant professional growth, as evidenced by her promotion to a senior position which indicates her ability to handle greater responsibilities and complex challenges. | When: Friday, December 12, 2025 | Involving: Alice (the professional being described), user (the one forming the opinion) | The user holds this belief because Alice's promotion to a senior position is seen as proof of her capability to take on more complex challenges, reflecting her dedication and skill in her profession.
  - The user formed the opinion that Alice's recent promotion and notable technical achievements indicate that she is working in a dynamic environment characterized by high stakes and innovative projects, which in turn reinforces the user's view of her strong career trajectory. | When: Friday, December 12, 2025 | Involving: Alice (the subject of the opinion as inferred from the context 'What 

In [29]:
# Temporal recall
results = client.recall(bank_id="my-bank", query="What happened in June?")

print("Memories:")
for r in results.results:
    print(f"  - {r.text}")

Memories:
  - Alice got promoted to senior engineer, marking an important career growth milestone with a change in her professional status. | When: Sunday, June 15, 2025 | Involving: Alice (an employee undergoing a career update) | Alice's promotion to senior engineer signifies a significant advancement in her career, reflecting her skills, performance, and the growth opportunities provided by her organization.
  - The user formed the opinion that Alice's recent promotion and notable technical achievements indicate that she is working in a dynamic environment characterized by high stakes and innovative projects, which in turn reinforces the user's view of her strong career trajectory. | When: Friday, December 12, 2025 | Involving: Alice (the subject of the opinion as inferred from the context 'What should I know about Alice?') | The user expressed this opinion to highlight the significance of Alice's promotion and technical accomplishments as evidence of her ability to thrive in challe

## Reflect: Generate Insights

The `reflect` operation performs a more thorough analysis of existing memories. This allows the agent to form new connections between memories which are then persisted as opinions and/or observations.

Example use cases:
- An AI Project Manager reflecting on what risks need to be mitigated
- A Sales Agent reflecting on why certain outreach messages have gotten responses
- A Support Agent reflecting on opportunities where customers have unanswered questions

In [30]:
response = client.reflect(bank_id="my-bank", query="What should I know about Alice?")
print(response)

text="I think you should know that Alice is a highly capable and dedicated professional at Google. Her career has shown impressive growth, evidenced by her recent promotion to senior engineer—a clear indication that she can handle more complex challenges and responsibilities. She’s been working in an innovative, dynamic environment where technical excellence and strong performance are key, and her upward trajectory speaks volumes about her skills and commitment. Overall, it's clear that Alice is not only excellent at what she does now but is also well-positioned for continued success in her field." based_on=[]


## Memory Types

Hindsight organizes memory into four networks to mimic human memory:

- **World**: Facts about the world ("The stove gets hot")
- **Experiences**: Agent's own experiences ("I touched the stove and it really hurt")
- **Opinion**: Beliefs with confidence scores ("I shouldn't touch the stove again" - .99 confidence)
- **Observation**: Complex mental models derived by reflecting on facts and experiences

## Cleanup

Delete the bank created during this notebook:

In [31]:
import requests

response = requests.delete(f"{HINDSIGHT_API_URL}/v1/default/banks/my-bank")
print(f"Deleted my-bank: {response.json()}")

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x110573650>


Deleted my-bank: {'success': True, 'message': "Bank 'my-bank' and all associated data deleted successfully", 'deleted_count': 23}
